In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
import os
import random
from collections import defaultdict  # Import defaultdict
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import Subset
import matplotlib.pyplot as plt
import wandb
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class FineTunedResNet(nn.Module):
    def __init__(self, num_classes, dense_neurons, activation='relu', batch_normalization=False, dropout_value=0, freeze_option=None,freeze_index=0):
        super(FineTunedResNet, self).__init__()
        
        # Load pre-trained ResNet model
        resnet = models.resnet50(pretrained=True)
        
        # Determine the number of layers in the ResNet model
        num_layers = len(list(resnet.children()))
        
        # Freeze layers based on the specified option
        if freeze_option == 'all_except_last':
            for param in resnet.parameters():
                param.requires_grad = False
            for param in resnet.fc.parameters():
                param.requires_grad = True
                
        elif freeze_option == 'up_to':
            # Freeze layers up to a certain index (excluding the last layer)
            if isinstance(freeze_index, int) and freeze_index < num_layers - 1:
                for idx, child in enumerate(resnet.children()):
                    if idx < freeze_index:
                        for param in child.parameters():
                            param.requires_grad = False
                    else:
                        break
            else:
                raise ValueError("Invalid 'freeze_index' value. It should be an integer less than the number of layers.")
                
        elif freeze_option == 'except_first':
            # Freeze all layers except the first convolutional layer
            for idx, child in enumerate(resnet.children()):
                if idx == 0:
                    continue
                for param in child.parameters():
                    param.requires_grad = False
                    
                    
        elif freeze_option== 'fine_tuning':
            pass
        
        # Modify the last fully connected layer to match the number of output features
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, dense_neurons)
        
        # Conditionally add batch normalization
        if batch_normalization:
            self.batch_norm = nn.BatchNorm1d(dense_neurons)
        else:
            self.batch_norm = None
        
        # Conditionally add dropout
        if dropout_value > 0:
            self.dropout = nn.Dropout(dropout_value)
        else:
            self.dropout = None
        
        # Activation function
        if activation == 'relu':
            self.act = nn.ReLU()
        elif activation == 'gelu':
            self.act = nn.GELU()
        elif activation == 'silu':
            self.act = nn.SiLU()
        elif activation == 'mish':
            self.act = Mish()
        else:
            raise ValueError("Invalid activation function specified.")
        
        # Add a new fully connected layer for the number of classes
        self.fc = nn.Linear(dense_neurons, num_classes)
        
        self.resnet = resnet

    def forward(self, x):
        x = self.resnet(x)
        
        if self.batch_norm is not None:
            x = self.batch_norm(x)
            x = self.act(x)
        
        if self.dropout is not None:
            x = self.dropout(x)
        
        x = self.fc(x)
        return x

class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))



In [3]:
def data_loader(train_data_folder,batch_size,data_augmentation):
    
    
    without_augmentation_transform =  transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    # Load the original training dataset
    train_dataset = datasets.ImageFolder(root=train_data_folder, transform=without_augmentation_transform)

    # Shuffle the dataset
    indices = list(range(len(train_dataset)))
    np.random.shuffle(indices)
    
    # Calculate the size of the validation set (20% of the training data)
    val_size = int(0.2 * len(train_dataset))
    
    # Calculate the number of samples per class for validation
    num_classes = len(train_dataset.classes)
    val_size_per_class = val_size // num_classes
    
    # Initialize lists to store indices for training and validation
    train_indices = []
    val_indices = []
    
    # Iterate through each class to select validation samples
    for class_idx in range(num_classes):
        class_indices = [i for i in indices if train_dataset.targets[i] == class_idx]
        val_indices.extend(class_indices[:val_size_per_class])
        train_indices.extend(class_indices[val_size_per_class:])
    


    if data_augmentation:

        # Define data augmentation transforms for training data
        train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224, 224)),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        # Create PyTorch data loaders for the initial dataset
        train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=batch_size, shuffle=True)
        
        
        transformed_dataset = datasets.ImageFolder(root=train_data_folder, transform=train_transform)
        transformed_loader = DataLoader(Subset(transformed_dataset, train_indices), batch_size=batch_size, shuffle=True)
        




  
        


        # Concatenate transformed datasets
        combined_train_dataset = ConcatDataset([train_loader.dataset,transformed_loader.dataset ])  # You can repeat train_dataset_transformed multiple times as needed
        
        # Define data loaders for combined datasets
        train_loader = DataLoader(dataset=combined_train_dataset, batch_size=batch_size, shuffle=True)
    

    else:
        # Create PyTorch data loaders for the initial dataset
        train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=batch_size, shuffle=True)


    
    return train_loader , val_loader

In [4]:

# Define training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    model.to(device)
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    return train_loss, train_accuracy

# Define testing function
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    model.to(device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(val_loader)
    val_accuracy = correct / total
    return val_loss, val_accuracy


In [5]:
def arguments(num_filters,batch_size,activation,filter_organization,batch_normalization,data_augmentation,dropout_value,dense_neurons,lr,num_epochs,freeze_option,freeze_index):
    


    # Set random seed
    torch.manual_seed(42)
    
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_data_folder='/kaggle/input/inaturalist/inaturalist_12K/train'

    train_dataset = datasets.ImageFolder(root=train_data_folder)
    


    

    # Model parameters
    input_channels = 3
    num_classes = len(train_dataset.classes)







    filter_size=3
    
    if(batch_normalization == "Yes"):
        batch_normalization_val=True
    elif(batch_normalization == "No"):
        batch_normalization_val=False

   



    
    
    
    if(data_augmentation == "Yes"):
        data_augmentation_val=True
    elif(data_augmentation == "No"):
        data_augmentation_val=False
    
    train_loader , val_loader=data_loader(train_data_folder,batch_size,data_augmentation_val)

    # Create model instance
    model = FineTunedResNet(num_classes, dense_neurons, activation, batch_normalization_val, dropout_value,freeze_option,freeze_index)
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr)



    # Training loop
  
    for epoch in range(num_epochs):
        # Initialize tqdm progress bars for training and validation
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Training', leave=False)

        # Training loop
        train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)

        # Validation loop
        val_loss, val_accuracy = validate(model, val_loader, criterion, device)


        # Log to Weights & Biases
        wandb.log({
            "Epoch": epoch + 1,
            "Train_Accuracy": train_accuracy,
            "Train_Loss": train_loss,
            "Val_Accuracy": val_accuracy,
            "Val_Loss": val_loss
        })

        # Print epoch results
        print(f"Epoch {epoch+1}/{num_epochs},\n Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f},\n Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

In [6]:
wandb.login()

wandb: Currently logged in as: cs23m046. Use `wandb login --relogin` to force relogin


True

In [7]:
# sweep_config = {
#     'method': 'bayes',
#     'name'  : 'Train Dataset Run - 3',
#     'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
#     'parameters': {
#         'num_filters': {'values': [32, 64, 128]},
#         'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
#         'filter_organization': {'values': ['same', 'double', 'halve']},
#         'batch_normalization': {'values': ['Yes', 'No']},
#         'dropout_value': {'values': [0.2, 0.3]},
#         'learning_rate': {'values': [0.001, 0.0001]},
#         'num_epochs': {'values': [5,10]},
#         'dense_neurons': {'values': [128, 256, 512, 1024]},
#         'batch_size': {'values': [32, 64]},
#         'data_augmentation': {'values': ['Yes', 'No']}
#     }
# }

# sweep_config = {
#     'method': 'bayes',
#     'name'  : 'Train Dataset Part B',
#     'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
#     'parameters': {
#         'num_filters': {'values': [32, 64, 128]},
#         'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
#         'filter_organization': {'values': ['same', 'double', 'halve']},
#         'batch_normalization': {'values': ['Yes']},
#         'dropout_value': {'values': [0.2, 0.3]},
#         'learning_rate': {'values': [0.001, 0.0001]},
#         'num_epochs': {'values': [10]},
#         'dense_neurons': {'values': [128, 256, 512, 1024]},
#         'batch_size': {'values': [32, 64]},
#         'freeze_option': {'values': ['all_except_last' ,'except_first','up_to']},
#         'freeze_index': {'values': [2,3,4]},
#         'data_augmentation': {'values': ['Yes']}
#     }
# }

sweep_config = {
    'method': 'bayes',
    'name'  : 'Train Dataset Part B',
    'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
    'parameters': {
        'num_filters': {'values': [128]},
        'activation': {'values': [gelu', 'mish']},
        'filter_organization': {'values': ['same', 'double']},
        'batch_normalization': {'values': ['Yes']},
        'dropout_value': {'values': [0.3]},
        'learning_rate': {'values': [0.0001]},
        'num_epochs': {'values': [10]},
        'dense_neurons': {'values': [1024]},
        'batch_size': {'values': [32]},
        'freeze_option': {'values': ['all_except_last' ,'except_first','up_to']},
        'freeze_index': {'values': [2,3,4]},
        'data_augmentation': {'values': ['Yes']}
    }
}

# sweep_config = {
#     'method': 'grid',
#     'name'  : 'Train Dataset Run - 2',
#     'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
#     'parameters': {
#         'num_filters': {'values': [128]},
#         'activation': {'values': ['gelu', 'mish']},
#         'filter_organization': {'values': [ 'double', 'halve']},
#         'batch_normalization': {'values': ['Yes']},
#         'dropout_value': {'values': [ 0.3]},
#         'learning_rate': {'values': [0.0001]},
#         'num_epochs': {'values': [10]},
#         'dense_neurons': {'values': [ 1024]},
#         'batch_size': {'values': [32]},
#         'data_augmentation': {'values': ['Yes']}
#     }
# }

# sweep_config = {
#     'method': 'random',
#     'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
#     'parameters': {
#         'num_filters': {'values': [32,64,128]},
#         'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
#         'filter_organization': {'values': ['same', 'double', 'halve']},
#         'batch_normalization': {'values': ['Yes', 'No']},
#         'dropout_value': {'values': [0.2,0.3]},
#         'learning_rate': {'values': [0.001,0.0001]},
#         'num_epochs': {'values': [5]},
#         'dense_neurons': {'values': [128, 256,512]},
#         'batch_size': {'values': [32,64]},
#         'data_augmentation': {'values': ['Yes', 'No']}
#     }
# }






# Create sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="DL_Assignment_2_CS23M046")



Create sweep with ID: ez9v2rtk
Sweep URL: https://wandb.ai/cs23m046/DL_Assignment_2_CS23M046/sweeps/ez9v2rtk


In [8]:
def main():
    
    # Initialize wandb
    with wandb.init() as run:

        config = wandb.config
        
        run_name="Resnet-activation_"+str(config.activation)+"_freeze_"+str(config.freeze_option)+"_num_filters_"+str(config.num_filters)+"_dense_"+str(config.dense_neurons)+"_lr_"+str(config.learning_rate)
        wandb.run.name=run_name
        
        arguments(config.num_filters,config.batch_size,config.activation,config.filter_organization,config.batch_normalization,config.data_augmentation,config.dropout_value,config.dense_neurons,config.learning_rate,config.num_epochs,config.freeze_option,config.freeze_index)






# Run sweep
wandb.agent(sweep_id, function=main, count=5)

wandb.finish()

wandb: Agent Starting Run: ghmvaprr with config:
wandb: 	activation: relu
wandb: 	batch_normalization: Yes
wandb: 	batch_size: 64
wandb: 	data_augmentation: Yes
wandb: 	dense_neurons: 128
wandb: 	dropout_value: 0.2
wandb: 	filter_organization: halve
wandb: 	freeze_index: 3
wandb: 	freeze_option: except_first
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: 	num_filters: 32


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10 - Training:   0%|          | 0/251 [00:00<?, ?it/s]

Epoch 1/10,
 Train Loss: 0.9158, Train Accuracy: 0.7068,
 Val Loss: 0.8869, Val Accuracy: 0.7035


Epoch 2/10,
 Train Loss: 0.7186, Train Accuracy: 0.7606,
 Val Loss: 0.8536, Val Accuracy: 0.7281


Epoch 3/10 - Training:   0%|          | 0/251 [00:00<?, ?it/s]
                                                              

...

Epoch 9/10 - Training:   0%|          | 0/251 [00:00<?, ?it/s]
                                                              

Epoch 9/10,
 Train Loss: 0.3787, Train Accuracy: 0.8883,
 Val Loss: 0.6969, Val Accuracy: 0.7819



                                                              ]
                                                               

Epoch 10/10,
 Train Loss: 0.3569, Train Accuracy: 0.8919,
 Val Loss: 0.6524, Val Accuracy: 0.7864


Epoch,▁▂▃▃▄▅▆▆▇█
Train_Accuracy,▁▄▅▅▆▆▇▇██
Train_Loss,█▅▄▃▃▂▂▂▁▁
Val_Accuracy,▁▃▄▆▃█▄▆▅▆
Val_Loss,█▅▄▁▅▁▂▂▄▁
Epoch,10
Train_Accuracy,0.89193
Train_Loss,0.35685
Val_Accuracy,0.78643
Val_Loss,0.65242
